In [42]:
import requests 
import pandas as pd 
import numpy as np 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [43]:
ClIENT_ID = 'NIFMB2XU3U2H3LWRBPZ1TOOCAIR2GVRDHRO5ZI4CTNFBQJXU'
ClIENT_SECRET = 'GU34LBGBFOWO0GHQIDSAKGM3TUQVWZFPDE52QQZVDOTQOTJI' 
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: NIFMB2XU3U2H3LWRBPZ1TOOCAIR2GVRDHRO5ZI4CTNFBQJXU
Foursquare_Secret:GU34LBGBFOWO0GHQIDSAKGM3TUQVWZFPDE52QQZVDOTQOTJI


In [44]:
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [45]:
search_query = 'Hotel'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NIFMB2XU3U2H3LWRBPZ1TOOCAIR2GVRDHRO5ZI4CTNFBQJXU&client_secret=GU34LBGBFOWO0GHQIDSAKGM3TUQVWZFPDE52QQZVDOTQOTJI&ll=43.6534817,-79.3839347&v=20180604&query=Hotel&radius=500&limit=30'

In [46]:
results = requests.get(url).json()


In [47]:

venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ab2d511f964a5209b6c20e3,123 Queen Street West,CA,Toronto,Canada,at York St.,324,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,Sheraton Centre Toronto Hotel,v-1591322281,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,51d212c3498ebf27dc469bc9,33 Gerrard Street West,CA,Toronto,Canada,at Yonge St,562,"[33 Gerrard Street West (at Yonge St), Toronto...","[{'label': 'display', 'lat': 43.65849759157591...",43.658498,-79.383097,NaN,M5G 1Z4,ON,Chelsea Hotel,v-1591322281,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ae61cf6f964a520caa421e3,200 Victoria St,CA,Toronto,Canada,at Shuter St,410,"[200 Victoria St (at Shuter St), Toronto ON, C...","[{'label': 'display', 'lat': 43.65449797039222...",43.654498,-79.379035,NaN,NaN,ON,Pantages Hotel & Spa,v-1591322281,NaN
3,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",False,4b68aed1f964a520de862be3,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,499,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,The Rex Hotel Jazz & Blues Bar,v-1591322281,62225795
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,51e48697498eded9073c6c17,33 Gerrard Street West,CA,Toronto,Canada,NaN,522,"[33 Gerrard Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65809435580944...",43.658094,-79.382711,NaN,NaN,ON,Crew Room Eaton Chelsea Hotel,v-1591322281,NaN


In [48]:

clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,CA,Toronto,Canada,at York St.,324,"[123 Queen Street West (at York St.), Toronto ...","[{'label': 'display', 'lat': 43.6505944, 'lng'...",43.650594,-79.384530,NaN,M5H 2M9,ON,4ab2d511f964a5209b6c20e3
1,Chelsea Hotel,Hotel,33 Gerrard Street West,CA,Toronto,Canada,at Yonge St,562,"[33 Gerrard Street West (at Yonge St), Toronto...","[{'label': 'display', 'lat': 43.65849759157591...",43.658498,-79.383097,NaN,M5G 1Z4,ON,51d212c3498ebf27dc469bc9
2,Pantages Hotel & Spa,Hotel,200 Victoria St,CA,Toronto,Canada,at Shuter St,410,"[200 Victoria St (at Shuter St), Toronto ON, C...","[{'label': 'display', 'lat': 43.65449797039222...",43.654498,-79.379035,NaN,NaN,ON,4ae61cf6f964a520caa421e3
3,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,CA,Toronto,Canada,Queen & St. Patrick,499,"[194 Queen St W (Queen & St. Patrick), Toronto...","[{'label': 'display', 'lat': 43.65050475544005...",43.650505,-79.388577,NaN,M5V 1Z1,ON,4b68aed1f964a520de862be3
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,CA,Toronto,Canada,NaN,522,"[33 Gerrard Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65809435580944...",43.658094,-79.382711,NaN,NaN,ON,51e48697498eded9073c6c17


In [49]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
2,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
3,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
4,Crew Room Eaton Chelsea Hotel,Hotel,33 Gerrard Street West,43.658094,-79.382711,NaN,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
8,Stathcona Hotel,Hotel,NaN,43.654947,-79.386359,NaN,ON
9,Sheraton Centre Hotel Club Lounge,Lounge,NaN,43.651063,-79.384527,M5H 2M9,ON


In [50]:
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
3,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
14,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
15,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
17,Pantages Hotel Gym,Gym,200 Victoria St.,43.654464,-79.379036,M5B 1V8,ON
18,Pantages Hotel Toronto Centre,Residential Building (Apartment / Condo),200 Victoria Street,43.655205,-79.379182,M5B 1V8,ON


In [51]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
14,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
15,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
25,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
27,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON


In [52]:
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
5,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
6,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
14,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
15,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
25,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
27,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON


In [53]:
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.38453,M5H 2M9,ON


In [54]:
search_query = 'Park'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NIFMB2XU3U2H3LWRBPZ1TOOCAIR2GVRDHRO5ZI4CTNFBQJXU&client_secret=GU34LBGBFOWO0GHQIDSAKGM3TUQVWZFPDE52QQZVDOTQOTJI&ll=43.6534817,-79.3839347&v=20180604&query=Park&radius=10000&limit=30'

In [55]:
presults = requests.get(url).json()

In [56]:
venues = presults['response']['venues']

park_dataframe = json_normalize(venues)
park_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4b170e47f964a5208cc123e3,691 University Ave,CA,Toronto,Canada,at College St,887,"[691 University Ave (at College St), Toronto O...","[{'label': 'display', 'lat': 43.66000635700919...",43.660006,-79.390272,M5G 2P1,ON,Queen's Park Subway Station,v-1591322282,NaN
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4b54deadf964a520a3d027e3,NaN,CA,Toronto,Canada,NaN,663,"[btwn College, University, Queen & Spadina, To...","[{'label': 'display', 'lat': 43.65248755299856...",43.652488,-79.392053,NaN,ON,Grange Park,v-1591322282,NaN
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bb79860cf2fc9b616779e02,35 Wellington St. East,CA,Toronto,Canada,NaN,929,"[35 Wellington St. East, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64804835388851...",43.648048,-79.375172,NaN,ON,Berczy Park,v-1591322282,NaN
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4b9d206bf964a520e69136e3,University Ave.,CA,Toronto,Canada,at Wellesley Ave.,1340,"[University Ave. (at Wellesley Ave.), Toronto ...","[{'label': 'display', 'lat': 43.66394609897775...",43.663946,-79.392180,M5R 2E8,ON,Queen's Park,v-1591322282,NaN
4,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",False,4b9177e4f964a520d2be33e3,NaN,CA,Toronto,Canada,NaN,389,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.650735, 'lng':...",43.650735,-79.386921,NaN,ON,TTC Streetcar #403 - Victoria park,v-1591322282,NaN


In [57]:
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Queen's Park Subway Station,Metro Station,691 University Ave,CA,Toronto,Canada,at College St,887,"[691 University Ave (at College St), Toronto O...","[{'label': 'display', 'lat': 43.66000635700919...",43.660006,-79.390272,M5G 2P1,ON,4b170e47f964a5208cc123e3
1,Grange Park,Park,NaN,CA,Toronto,Canada,NaN,663,"[btwn College, University, Queen & Spadina, To...","[{'label': 'display', 'lat': 43.65248755299856...",43.652488,-79.392053,NaN,ON,4b54deadf964a520a3d027e3
2,Berczy Park,Park,35 Wellington St. East,CA,Toronto,Canada,NaN,929,"[35 Wellington St. East, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.64804835388851...",43.648048,-79.375172,NaN,ON,4bb79860cf2fc9b616779e02
3,Queen's Park,Park,University Ave.,CA,Toronto,Canada,at Wellesley Ave.,1340,"[University Ave. (at Wellesley Ave.), Toronto ...","[{'label': 'display', 'lat': 43.66394609897775...",43.663946,-79.392180,M5R 2E8,ON,4b9d206bf964a520e69136e3
4,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,CA,Toronto,Canada,NaN,389,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.650735, 'lng':...",43.650735,-79.386921,NaN,ON,4b9177e4f964a520d2be33e3


In [58]:
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
1,Grange Park,Park,NaN,43.652488,-79.392053,NaN,ON
2,Berczy Park,Park,35 Wellington St. East,43.648048,-79.375172,NaN,ON
3,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
4,TTC Streetcar #403 - Victoria park,Light Rail Station,NaN,43.650735,-79.386921,NaN,ON
5,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,43.647388,-79.398752,NaN,ON
6,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
7,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
8,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
9,Clarence Square Park,Park,Spadina Ave,43.644222,-79.394230,NaN,ON


In [59]:
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,M5G 2P1,ON
3,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
6,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,M5T 2Y5,ON
7,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,M5B 1S8,ON
8,Crunch Fitness - College Park,Gym / Fitness Center,382 Yonge Street,43.659263,-79.382632,M5G 1S8,ON
10,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
11,Residences at College Park North,Residential Building (Apartment / Condo),763 Bay St.,43.659822,-79.385159,M5G 2R3,ON
13,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
14,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
16,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON


In [60]:
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng,postalCode,state
3,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
10,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
13,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
14,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
16,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
23,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON
26,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON


In [61]:
search_query = 'Restaurant'
radius = 10000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NIFMB2XU3U2H3LWRBPZ1TOOCAIR2GVRDHRO5ZI4CTNFBQJXU&client_secret=GU34LBGBFOWO0GHQIDSAKGM3TUQVWZFPDE52QQZVDOTQOTJI&ll=43.6534817,-79.3839347&v=20180604&query=Restaurant&radius=10000&limit=30'

In [62]:
Rresults = requests.get(url).json()

In [63]:
venues = Rresults['response']['venues']

Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4ad4c05ff964a52048f720e3,110 Chestnut Street,CA,Toronto,Canada,NaN,224,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,Hemispheres Restaurant & Bistro,v-1591322283,NaN
1,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4ad4c05cf964a52006f620e3,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,763,[37 King Street East (at Le Meridien King Edwa...,"[{'label': 'display', 'lat': 43.64929834396347...",43.649298,-79.376431,NaN,M5C 1E9,ON,Victoria's Restaurant,v-1591322283,498556908
2,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",False,5750b013498e755287c6de97,988 Baldwin Street,CA,Toronto,Canada,NaN,839,"[988 Baldwin Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655874, 'lng':...",43.655874,-79.393826,NaN,NaN,ON,Some Time BBQ Grill Restaurant 碳烤屋,v-1591322283,NaN
3,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b074bb1f964a52077fb22e3,353 Spadina Ave.,CA,Toronto,Canada,NaN,1222,"[353 Spadina Ave., Toronto ON M5T 2G3, Canada]","[{'label': 'display', 'lat': 43.65533674412141...",43.655337,-79.398897,NaN,M5T 2G3,ON,New Sky Restaurant 小沙田食家,v-1591322283,NaN
4,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4aef8854f964a5201cd921e3,287 King St. W,CA,Toronto,Canada,at John St.,906,"[287 King St. W (at John St.), Toronto ON M5V ...","[{'label': 'display', 'lat': 43.64646252150344...",43.646463,-79.389644,NaN,M5V 1J5,ON,Aroma Fine Indian Restaurant,v-1591322283,NaN


In [64]:

Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,CA,Toronto,Canada,NaN,224,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,NaN,M5G 1R3,ON,4ad4c05ff964a52048f720e3
1,Victoria's Restaurant,Restaurant,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,763,[37 King Street East (at Le Meridien King Edwa...,"[{'label': 'display', 'lat': 43.64929834396347...",43.649298,-79.376431,NaN,M5C 1E9,ON,4ad4c05cf964a52006f620e3
2,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,CA,Toronto,Canada,NaN,839,"[988 Baldwin Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655874, 'lng':...",43.655874,-79.393826,NaN,NaN,ON,5750b013498e755287c6de97
3,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,CA,Toronto,Canada,NaN,1222,"[353 Spadina Ave., Toronto ON M5T 2G3, Canada]","[{'label': 'display', 'lat': 43.65533674412141...",43.655337,-79.398897,NaN,M5T 2G3,ON,4b074bb1f964a52077fb22e3
4,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,CA,Toronto,Canada,at John St.,906,"[287 King St. W (at John St.), Toronto ON M5V ...","[{'label': 'display', 'lat': 43.64646252150344...",43.646463,-79.389644,NaN,M5V 1J5,ON,4aef8854f964a5201cd921e3


In [65]:
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
2,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,NaN,ON
3,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON
4,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,M5V 1J5,ON
5,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
6,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
7,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,M5V 1H3,ON
8,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
9,North-East Chinese Restaurant 華北美食,Chinese Restaurant,476 Dundas St.,43.653185,-79.396677,NaN,ON


In [66]:
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
3,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON
4,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,M5V 1J5,ON
5,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,43.654318,-79.398650,M5T 2E9,ON
6,Azure Restaurant & Bar,Restaurant,225 Front St W,43.644749,-79.385113,M5V 2X3,ON
7,Victor Restaurant & Bar,Bar,30 Mercer Street,43.645634,-79.391125,M5V 1H3,ON
8,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
10,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON
11,Green Tea Restaurant Downtown,Chinese Restaurant,261 Spadina Avenue. Upper level,43.652488,-79.397501,M5T 2E3,ON


In [67]:
search_query = 'Cafeteria'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NIFMB2XU3U2H3LWRBPZ1TOOCAIR2GVRDHRO5ZI4CTNFBQJXU&client_secret=GU34LBGBFOWO0GHQIDSAKGM3TUQVWZFPDE52QQZVDOTQOTJI&ll=43.6534817,-79.3839347&v=20180604&query=Cafeteria&radius=10000&limit=30'

In [68]:
cresults = requests.get(url).json()

In [69]:
venues = cresults['response']['venues']

Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4f9165bde4b04fef0087e4e2,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,448,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,Cafeteria,v-1591322283
1,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,528e99a211d262edc3708a6f,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,1246,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,Stay Cafeteria 慢走,v-1591322283
2,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,4b57256bf964a520332828e3,Yonge,CA,Toronto,Canada,Dundas,387,"[Yonge (Dundas), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65603785909391...",43.656038,-79.380672,NaN,ON,Sears Cafeteria,v-1591322283
3,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,53bd6dfe498e6c1d3bc9333e,NaN,CA,Toronto,Canada,NaN,505,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,Industrial Alliance Cafeteria,v-1591322283
4,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,524f1c8f498e983dde61ada0,89 Chestnut Street,CA,Toronto,Canada,NaN,584,"[89 Chestnut Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.648231, 'lng':...",43.648231,-79.384045,NaN,ON,Chestnut Tree Cafeteria,v-1591322283


In [70]:
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),CA,Toronto,Canada,NaN,448,"[The Hospital for Sick Children (SickKids), To...","[{'label': 'display', 'lat': 43.65720930549997...",43.657209,-79.386063,NaN,ON,4f9165bde4b04fef0087e4e2
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,CA,Toronto,Canada,at Nassau St,1246,"[388 Spadina Ave (at Nassau St), Toronto ON M5...","[{'label': 'display', 'lat': 43.65545427446646...",43.655454,-79.399163,M5T 2G5,ON,528e99a211d262edc3708a6f
2,Sears Cafeteria,Food Court,Yonge,CA,Toronto,Canada,Dundas,387,"[Yonge (Dundas), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65603785909391...",43.656038,-79.380672,NaN,ON,4b57256bf964a520332828e3
3,Industrial Alliance Cafeteria,Cafeteria,NaN,CA,Toronto,Canada,NaN,505,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.655657, 'lng':...",43.655657,-79.389443,NaN,ON,53bd6dfe498e6c1d3bc9333e
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,CA,Toronto,Canada,NaN,584,"[89 Chestnut Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.648231, 'lng':...",43.648231,-79.384045,NaN,ON,524f1c8f498e983dde61ada0


In [71]:
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,NaN,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,NaN,ON
3,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,NaN,ON
4,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,NaN,ON
5,OCAD Cafeteria,College Cafeteria,100 McCaul St.,43.652780,-79.391762,NaN,ON
6,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN
7,Ryerson Hub Cafeteria,College Cafeteria,350 Victoria St.,43.658475,-79.377653,NaN,ON
8,SickKids Cafeteria,Food Court,555 University Avenue,43.657584,-79.375652,NaN,ON
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON


In [72]:
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,postalCode,state
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
9,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
17,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
20,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
24,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,M5S 1J5,ON
28,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON


In [73]:
search_query = 'Shopping'
radius = 1000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NIFMB2XU3U2H3LWRBPZ1TOOCAIR2GVRDHRO5ZI4CTNFBQJXU&client_secret=GU34LBGBFOWO0GHQIDSAKGM3TUQVWZFPDE52QQZVDOTQOTJI&ll=43.6534817,-79.3839347&v=20180604&query=Shopping&radius=1000&limit=30'

In [74]:
sresults = requests.get(url).json()

In [75]:
venues = sresults['response']['venues']
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,5840cafe06f1a34af80cc609,176 Yonge Street,CA,Toronto,Canada,Queen Street West,424,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,Saks Fifth Avenue Club - Personal Shopping,v-1591322284
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b9e86dff964a520e6eb36e3,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,741,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,TD Centre Shopping Concourse,v-1591322284
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4ad77a12f964a520260b21e3,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,287,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.654540,-79.380677,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,CF Toronto Eaton Centre,v-1591322284


In [76]:
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,CA,Toronto,Canada,Queen Street West,424,"[176 Yonge Street (Queen Street West), Toronto...","[{'label': 'display', 'lat': 43.65181, 'lng': ...",43.651810,-79.379192,NaN,M5C 2L7,ON,5840cafe06f1a34af80cc609
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,CA,Toronto,Canada,btw York & Bay St,741,"[66 Wellington St W (btw York & Bay St), Toron...","[{'label': 'display', 'lat': 43.64718424611984...",43.647184,-79.380932,NaN,M5K 1A1,ON,4b9e86dff964a520e6eb36e3
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,287,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.654540,-79.380677,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,4ad77a12f964a520260b21e3


In [77]:
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


In [78]:
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


In [79]:
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Stay Cafeteria 慢走,Asian Restaurant,388 Spadina Ave,43.655454,-79.399163,M5T 2G5,ON
2,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
3,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
4,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
5,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,M5S 1J5,ON
6,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON
7,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
8,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


In [80]:
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

In [81]:
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Queen's Park,Park,University Ave.,43.663946,-79.392180,M5R 2E8,ON
1,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,M5H 2N2,ON
2,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,M5A 2H3,ON
3,Riverdale Park East,Park,550 Broadview Ave,43.669951,-79.355493,M4K 2P1,ON
4,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,M5T 2N4,ON
5,Coronation Park,Park,711 Lake Shore Blvd. W,43.634553,-79.405403,M5V 3T7,ON
6,High Park,Park,1873 Bloor St. W,43.646479,-79.463425,M6R 2Z3,ON
7,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
8,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
9,New Sky Restaurant 小沙田食家,Chinese Restaurant,353 Spadina Ave.,43.655337,-79.398897,M5T 2G3,ON


In [82]:
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    